In [1]:
import tensorflow as tf
import numpy as np

#mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
n_input = 28 # MNIST data 输入 (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10  # MNIST 列别 (0-9 ，一共10类)
batch_size = 128

In [3]:
tf.reset_default_graph()
#
x = tf.placeholder("float",[None,n_steps,n_input])
y = tf.placeholder("float",[None,n_classes])

In [4]:
gru = tf.contrib.rnn.GRUCell(n_hidden*2)
lstm_cell = tf.contrib.rnn.LSTMCell(n_hidden)
mcell = tf.contrib.rnn.MultiRNNCell([lstm_cell,gru])

In [5]:
x1 = tf.unstack(x, n_steps, 1)
outputs, states = tf.contrib.rnn.static_rnn(mcell, x1, dtype=tf.float32)
pred = tf.contrib.layers.fully_connected(outputs[-1],n_classes,activation_fn = None)

In [7]:
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 1
    while step * batch_size < training_iters:
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size,n_steps,n_input))
        sess.run(optimizer,feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # 计算批次数据的准确率
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print (" Finished!")

    # 计算准确率 for 128 mnist test images
    test_len = 100
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 1.924278, Training Accuracy= 0.33594
Iter 2560, Minibatch Loss= 1.344485, Training Accuracy= 0.54688
Iter 3840, Minibatch Loss= 1.132469, Training Accuracy= 0.63281
Iter 5120, Minibatch Loss= 0.915656, Training Accuracy= 0.70312
Iter 6400, Minibatch Loss= 0.947829, Training Accuracy= 0.66406
Iter 7680, Minibatch Loss= 0.674243, Training Accuracy= 0.78906
Iter 8960, Minibatch Loss= 0.535811, Training Accuracy= 0.81250
Iter 10240, Minibatch Loss= 0.581722, Training Accuracy= 0.82031
Iter 11520, Minibatch Loss= 0.437268, Training Accuracy= 0.83594
Iter 12800, Minibatch Loss= 0.374421, Training Accuracy= 0.87500
Iter 14080, Minibatch Loss= 0.348179, Training Accuracy= 0.85938
Iter 15360, Minibatch Loss= 0.369004, Training Accuracy= 0.86719
Iter 16640, Minibatch Loss= 0.139150, Training Accuracy= 0.96875
Iter 17920, Minibatch Loss= 0.264871, Training Accuracy= 0.89062
Iter 19200, Minibatch Loss= 0.295519, Training Accuracy= 0.89844
Iter 20480, Minibatch Loss= 0.23